# TARS Repo Functional Test Notebook

This Colab notebook validates the repository setup, runs tests, and executes both an offline mock analysis and an optional live Gemini analysis.

## 1) Clone and install
Run this first in Colab.

In [ ]:
!git clone https://github.com/<your-org-or-user>/tars.git
%cd tars
!python -m pip install --upgrade pip
!pip install -e .

## 2) Run unit tests

In [ ]:
!python -m unittest discover -s tests

## 3) Offline functionality test (no Gemini key needed)
This monkeypatches `GeminiEvaluator.evaluate_progression` so you can validate sequence-level self-improvement reporting.

In [ ]:
import json
from pathlib import Path

from tars_analyzer import analyzer
from tars_analyzer.models import ConversationProgress, ProgressionEvaluation

class FakeEvaluator:
    def __init__(self, model='gemini-2.0-flash'):
        self.model = model

    def evaluate_progression(self, conversations):
        ordered = sorted(conversations, key=lambda c: c.timestamp)
        items = []
        for idx, convo in enumerate(ordered):
            items.append(ConversationProgress(
                conversation_id=convo.conversation_id,
                rank=idx + 1,
                overall_agent_quality=6.0 + idx,
                improvement_vs_previous=0.0 if idx == 0 else 1.0,
                notes='mock sequence improvement'
            ))
        return ProgressionEvaluation(
            overall_summary='Agent improves each step in this mocked run.',
            trajectory_label='improving',
            trajectory_confidence=9.0,
            per_conversation=items
        )

sample_jsonl = '\n'.join([
    json.dumps({
        'conversation_id': 'conv-0',
        'timestamp': '2026-01-01T10:00:00Z',
        'turns': [
            {'role': 'human', 'content': 'Can you plan a trip?'},
            {'role': 'agent', 'content': 'Sure, here is a rough plan.'}
        ]
    }),
    json.dumps({
        'conversation_id': 'conv-1',
        'timestamp': '2026-01-02T10:00:00Z',
        'turns': [
            {'role': 'human', 'content': 'Can you improve the budget detail?'},
            {'role': 'agent', 'content': 'Absolutely, here is a detailed budget with assumptions.'}
        ]
    }),
    json.dumps({
        'conversation_id': 'conv-2',
        'timestamp': '2026-01-03T10:00:00Z',
        'turns': [
            {'role': 'human', 'content': 'Add risk mitigation and alternatives.'},
            {'role': 'agent', 'content': 'Added fallback options, risks, and trade-offs.'}
        ]
    })
])

Path('tmp').mkdir(exist_ok=True)
input_path = Path('tmp/mock_conversations.jsonl')
output_dir = Path('tmp/mock_output')
input_path.write_text(sample_jsonl)

original = analyzer.GeminiEvaluator
analyzer.GeminiEvaluator = FakeEvaluator
try:
    report = analyzer.analyze_conversations(input_path, output_dir)
finally:
    analyzer.GeminiEvaluator = original

print('Trajectory:', report['trajectory']['label'])
print('First→Last delta:', report['trend_delta_first_to_last'])
print('Report files:', list(output_dir.iterdir()))

In [ ]:
!cat tmp/mock_output/report.md

## 4) Optional: Live Gemini test
Run this only if you have a Gemini API key.

In [ ]:
import os
from getpass import getpass

if not os.getenv('GEMINI_API_KEY'):
    os.environ['GEMINI_API_KEY'] = getpass('Enter GEMINI_API_KEY: ')

In [ ]:
!python -m tars_analyzer.cli examples/conversations.jsonl --out output --model gemini-2.0-flash
!cat output/report.md